# Numenta Anomaly Benchmark

In [1]:
import pandas as pd
import json
import os
from typing import Final
from collections.abc import Callable
from datetime import datetime
from config import data_raw_folder, data_processed_folder
from timeeval import Datasets

In [2]:
dataset_collection_name = "NAB"
source_folder = os.path.join(data_raw_folder, "Community-NAB")
target_folder = data_processed_folder

from pathlib import Path
print(f"Looking for source datasets in {Path(source_folder).absolute()} and\nsaving processed datasets in {Path(target_folder).absolute()}")

Looking for source datasets in /home/projects/akita/data/benchmark-data/data-raw/Community-NAB and
saving processed datasets in /home/projects/akita/data/benchmark-data/data-processed


In [3]:
def calc_size(filename: str) -> int:
    with open(filename, 'r') as f:
        next(f) # skips header
        c = 0
        for line in f:
            c += 1
    return c

def transform_and_label(source: str, target: str, anomaly_windows: list[str]) -> None:
    df = pd.read_csv(source)
    df["timestamp"] = pd.to_datetime(df['timestamp'], infer_datetime_format=True)
    df["is_anomaly"] = 0

    for t1, t2 in anomaly_windows:
        t1 = datetime.strptime(t1, "%Y-%m-%d %H:%M:%S.%f")
        t2 = datetime.strptime(t2, "%Y-%m-%d %H:%M:%S.%f")
        moreThanT1 = df[df["timestamp"] >= t1]
        betweenT1AndT2 = moreThanT1[moreThanT1["timestamp"] <= t2]
        indices = betweenT1AndT2.index
        df["is_anomaly"].values[indices.values] = 1

    df.to_csv(target, index=False)

In [5]:
# shared by all datasets
input_type = "univariate"
datetime_index = True
train_type = "unsupervised"
train_is_normal = False

# create target directory
dataset_subfolder = os.path.join(input_type, dataset_collection_name)
target_subfolder = os.path.join(target_folder, dataset_subfolder)
try:
    os.makedirs(target_subfolder)
    print(f"Created directories {target_subfolder}")
except FileExistsError:
    print(f"Directories {target_subfolder} already exist")
    pass

dm = Datasets(target_folder)

with open(os.path.join(source_folder, "labels", "combined_windows.json"), 'r') as f:
    windows = json.load(f)

#windows

Directories /home/projects/akita/data/benchmark-data/data-processed/univariate/NAB already exist


In [6]:
# dataset transformation
transform_file: Callable[[str, str, list[str]], None] = transform_and_label

for dataset in windows:
    source_file = os.path.join(source_folder, "data", dataset)
    dataset_type = "real" if dataset.startswith("real") else "synthetic"
    
    # get basename for target filename
    basename = os.path.splitext(os.path.basename(source_file))[0]
    filename = f"{basename}.test.csv"

    # save metadata
    dataset_name = filename.split(".")[0]
    path = os.path.join(dataset_subfolder, filename)
    target_filepath = os.path.join(target_subfolder, filename)
    dataset_length = calc_size(source_file)
    dm.add_dataset((dataset_collection_name, dataset_name),
        train_path = None,
        test_path = path,
        dataset_type = dataset_type,
        datetime_index = datetime_index,
        split_at = None,
        train_type = train_type,
        train_is_normal = train_is_normal,
        input_type = input_type,
        dataset_length = dataset_length
    )
    # transform file
    transform_file(source_file, target_filepath, windows[dataset])
    print(f"Processed source dataset {source_file} -> {target_filepath}")

# save metadata of benchmark
dm.save()

Processed source dataset /home/projects/akita/data/benchmark-data/data-raw/Community-NAB/data/artificialNoAnomaly/art_daily_no_noise.csv -> /home/projects/akita/data/benchmark-data/data-processed/univariate/NAB/art_daily_no_noise.test.csv
Processed source dataset /home/projects/akita/data/benchmark-data/data-raw/Community-NAB/data/artificialNoAnomaly/art_daily_perfect_square_wave.csv -> /home/projects/akita/data/benchmark-data/data-processed/univariate/NAB/art_daily_perfect_square_wave.test.csv
Processed source dataset /home/projects/akita/data/benchmark-data/data-raw/Community-NAB/data/artificialNoAnomaly/art_daily_small_noise.csv -> /home/projects/akita/data/benchmark-data/data-processed/univariate/NAB/art_daily_small_noise.test.csv
Processed source dataset /home/projects/akita/data/benchmark-data/data-raw/Community-NAB/data/artificialNoAnomaly/art_flatline.csv -> /home/projects/akita/data/benchmark-data/data-processed/univariate/NAB/art_flatline.test.csv
Processed source dataset /ho

In [7]:
dm.refresh()
dm.df().loc[(slice(dataset_collection_name,dataset_collection_name), slice(None))]

train_path  \
collection_name dataset_name                                      
NAB             TravelTime_387                              NaN   
                TravelTime_451                              NaN   
                Twitter_volume_AAPL                         NaN   
                Twitter_volume_AMZN                         NaN   
                Twitter_volume_CRM                          NaN   
                Twitter_volume_CVS                          NaN   
                Twitter_volume_FB                           NaN   
                Twitter_volume_GOOG                         NaN   
                Twitter_volume_IBM                          NaN   
                Twitter_volume_KO                           NaN   
                Twitter_volume_PFE                          NaN   
                Twitter_volume_UPS                          NaN   
                ambient_temperature_system_failure          NaN   
                art_daily_flatmiddle                        NaN   
                art_daily_jumpsdown                         NaN   
                art_daily_jumpsup                           NaN   
                art_daily_no_noise                          NaN   
                art_daily_nojump                            NaN   
                art_daily_perfect_square_wave               NaN   
                art_daily_small_noise                       NaN   
                art_flatline                                NaN   
                art_increase_spike_density                  NaN   
                art_load_balancer_spikes                    NaN   
                art_noisy                                   NaN   
                cpu_utilization_asg_misconfiguration        NaN   
                ec2_cpu_utilization_24ae8d                  NaN   
                ec2_cpu_utilization_53ea38                  NaN   
                ec2_cpu_utilization_5f5533                  NaN   
                ec2_cpu_utilization_77c1ca                  NaN   
                ec2_cpu_utilization_825cc2                  NaN   
                ec2_cpu_utilization_ac20cd                  NaN   
                ec2_cpu_utilization_c6585a                  NaN   
                ec2_cpu_utilization_fe7f93                  NaN   
                ec2_disk_write_bytes_1ef3de                 NaN   
                ec2_disk_write_bytes_c0d644                 NaN   
                ec2_network_in_257a54                       NaN   
                ec2_network_in_5abac7                       NaN   
                ec2_request_latency_system_failure          NaN   
                elb_request_count_8c0756                    NaN   
                exchange-2_cpc_results                      NaN   
                exchange-2_cpm_results                      NaN   
                exchange-3_cpc_results                      NaN   
                exchange-3_cpm_results                      NaN   
                exchange-4_cpc_results                      NaN   
                exchange-4_cpm_results                      NaN   
                grok_asg_anomaly                            NaN   
                iio_us-east-1_i-a2eb1cd9_NetworkIn          NaN   
                machine_temperature_system_failure          NaN   
                nyc_taxi                                    NaN   
                occupancy_6005                              NaN   
                occupancy_t4013                             NaN   
                rds_cpu_utilization_cc0c53                  NaN   
                rds_cpu_utilization_e47b3b                  NaN   
                rogue_agent_key_hold                        NaN   
                rogue_agent_key_updown                      NaN   
                speed_6005                                  NaN   
                speed_7578                                  NaN   
                speed_t4013                                 NaN   

                                

## Experimentation

In [ ]:
dataset = "realAdExchange/exchange-4_cpc_results.csv"
source_file = os.path.join(source_folder, "data", dataset)
df = pd.read_csv(source_file)
df["timestamp"] = pd.to_datetime(df['timestamp'], infer_datetime_format=True)
df["is_anomaly"] = 0

for t1, t2 in windows[dataset]:
    t1 = datetime.strptime(t1, "%Y-%m-%d %H:%M:%S.%f")
    t2 = datetime.strptime(t2, "%Y-%m-%d %H:%M:%S.%f")
    moreThanT1 = df[df["timestamp"] >= t1]
    betweenT1AndT2 = moreThanT1[moreThanT1["timestamp"] <= t2]
    indices = betweenT1AndT2.index
    df["is_anomaly"].values[indices.values] = 1

df[df["is_anomaly"] == 1]

In [ ]:
with open("data-raw/Community-NAB/labels/combined_labels.json", 'r') as f:
    labels = json.load(f)
labels

In [ ]:
with open("data-raw/Community-NAB/labels/combined_windows.json", 'r') as f:
    windows = json.load(f)
windows

In [ ]:
def to_datetime(str):
    try:
        return datetime.strptime(str, "%Y-%m-%d %H:%M:%S")
    except ValueError:
        return datetime.strptime(str, "%Y-%m-%d %H:%M:%S.%f")

In [ ]:
matches = 0
for dataset in windows:
    for (anomaly, anomaly_window) in zip(labels[dataset], windows[dataset]):
        maybe_middle = to_datetime(anomaly)
        (start, end) = [to_datetime(d) for d in anomaly_window]
        diff1 = maybe_middle - start
        diff2 = end - maybe_middle
        if diff1 == diff2:
            # print(f"{dataset}-{anomaly} is in the middle of anomaly window!")
            matches += 1
        else:
            print(dataset)
            print(f"{start} - ({diff1})- {anomaly} -({diff2})- {end}")
print(f"matches: {matches}/{sum(list(map(lambda x: len(labels[x]), labels)))}")